## Feature Extraction for Convolutional-Autoencoder 

Con los datos generados en 1.0, separamos en conjuntos de entrenamiento y testeo, y probamos tres normalizaciones distintas.

Luego visualizamos la distribución de los datos bajo cada una de las normalizaciones realizadas. 

Comenzamos importando los módulos necesarios y cargando los datos crudos.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

from deep_scattering_models.data.create_data import load_data

from deep_scattering_models.visualization.visualize import plot_polarization_signature, plot_histogram

from deep_scattering_models.features.preprocess_data import to_dB, RScaler, MMScaler


In [ ]:
file_name = "spm_signatures_no_noise.npy"
data = load_data(file_name)

### 1 - Train-Test split y Robust Scaler

Comenzamos separando en grupos de entrenamiento y testeo, para luego estandarizar los datos. En el estandarizado robusto, se resta la media de de todos los pixels, y se divide por el interquartile rate.

In [ ]:
train_set, test_set = train_test_split(
    data, 
    test_size=.2, 
    random_state=123
    )

print(f'Tamaño de los datos de entrenamiento: {train_set.shape}')
print(f'Tamaño de los datos de testeo: {test_set.shape}')

In [ ]:
# Fiteo con el conjunto de entrenamiento
robust_scaler = RScaler().fit(train_set)

# Transformo ambos conjuntos
rscaled_train = robust_scaler.transform(train_set)
rscaled_test = robust_scaler.transform(test_set)

Tomemos algún ejemplo al azar del conjunto de entrenamiento y visualicémoslo

In [ ]:
# Elegimos ana firma al azar
rng = np.random.default_rng()
rnd_index = rng.integers(20048)
rnd_signature = rscaled_train[rnd_index, :, :]

# Ploteamos
plot_polarization_signature(rnd_signature)